<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Agent/blob/main/LangChain/00-Memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -U langgraph langchain langchain_openai

In [4]:
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_openai import ChatOpenAI # Corrected import
import os

In [5]:
from google.colab import userdata
import openai
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
openai.api_key  = os.environ["OPENAI_API_KEY"]

In [6]:
llm = ChatOpenAI(model="gpt-4o-mini")

# 1) 상태 스키마 정의
workflow = StateGraph(state_schema=MessagesState)

# 2) 모델 호출 노드 정의: 이전 메시지 리스트 사용
def call_model(state: MessagesState):
    response = llm.invoke(state["messages"])
    return {"messages": response}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# 3) 메모리 저장소 (MemorySaver) 활용
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "user123"}}

# 첫 대화
output = app.invoke({"messages": [HumanMessage("안녕!") ]}, config)
print(output["messages"][-1])

# 이어지는 대화
output = app.invoke({"messages": [HumanMessage("내 이름이 뭐였지?") ]}, config)
print(output["messages"][-1])


content='안녕하세요! 어떻게 도와드릴까요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 10, 'total_tokens': 20, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CBrNJILE6d757edzUvsB3EDhn9Wq4', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--1ce97f48-575d-4ec8-8da7-a1a475370bb1-0' usage_metadata={'input_tokens': 10, 'output_tokens': 10, 'total_tokens': 20, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
content='죄송하지만, 이전 대화 내용을 기억할 수 없어서 이름을 알 수 없습니다. 당신의 이름을 알려주시면 기억할 수 있을 것 같아요!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 35,

In [7]:
# 이어지는 대화 with memory
print("\n--- 이어지는 대화 (메모리 사용) ---")
output = app.invoke({"messages": [HumanMessage("내 이름이 뭐였지?")]}, config)
print(output["messages"][-1])

# 메모리 확인 (optional)
# thread_id = config["configurable"]["thread_id"]
# saved_messages = memory.get(thread_id)
# print("\n--- 저장된 메모리 확인 ---")
# print(saved_messages)


--- 이어지는 대화 (메모리 사용) ---
content='죄송하지만, 저는 이전의 대화 내용을 기억할 수 없어서 당신의 이름을 알 수 없습니다. 당신의 이름을 말씀해 주시면 그에 맞춰 이야기할 수 있을 거예요!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 84, 'total_tokens': 127, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CBrObnjbUntf7jHvm8JfOndMeKqIu', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--880f2206-982d-45a8-8ddb-785a2ae587c8-0' usage_metadata={'input_tokens': 84, 'output_tokens': 43, 'total_tokens': 127, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
